In [ ]:
!pip install opencv-python pyzbar Pillow imagehash scikit-image requests
!pip install pyzbar
!pip install opencv-python pillow imagehash scikit-image requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 6.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### QR Code Alteration Detection Method

This method detects tampered QR codes by combining **payload verification** with **image similarity analysis**. It checks both the data encoded in the QR and the visual integrity of the QR image.

## 1. Payload Verification
- The QR code is decoded using OpenCV’s `QRCodeDetector`.
- Multiple strategies maximize readability:
  - **Direct decoding** of a single QR code.
  - **Multi-QR decoding** if multiple codes are present.
  - **Preprocessing** (grayscale conversion, histogram equalization, thresholding) for noisy or damaged images.
- **Purpose:** If the decoded payload differs from the reference QR, the QR is immediately flagged as suspicious.

## 2. Image Similarity Analysis
Even if the payload matches, the QR image itself might be visually altered. Three complementary techniques compare the candidate QR with a reference image:

1. **Perceptual Hash (pHash)**  
   - Summarizes the image into a compact hash representing its overall visual appearance.  
   - Measures global visual similarity: small differences → similar images, large differences → possible alteration.

2. **Structural Similarity Index (SSIM)**  
   - Measures similarity in **structure, contrast, and brightness**.  
   - Values range from 0 (different) to 1 (identical).  
   - Detects blurring, noise, or contrast/color changes.

3. **ORB Feature Matching**  
   - Detects **local keypoints** (corners, edges) and computes descriptors.  
   - Matches features between candidate and reference QR codes.  
   - Fewer good matches indicate local edits, scratches, or partial tampering.

## 3. Risk Scoring
- A simple scoring system combines all checks:
  - Payload mismatch → **50 points**  
  - pHash distance > 8 → **20 points**  
  - SSIM < 0.7 → **15 points**  
  - ORB score < 0.15 → **15 points**  
- **Total score ≥ 40 → QR is flagged as suspected altered.**



In [ ]:
import cv2
from PIL import Image
import imagehash
from skimage.metrics import structural_similarity as ssim
import os

# ------------------- QR DECODING -------------------
def decode_qr(image_path):
    """
    Decode a QR code from an image.
    Tries multiple strategies: direct, multi QR, and pre-processed thresholded image.
    Returns the decoded payload string or None if not readable.
    """
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Cannot read image: {image_path}")

    detector = cv2.QRCodeDetector()
    data, points, _ = detector.detectAndDecode(img)
    if data:
        return data

    try:
        ok, decoded_info, _, _ = detector.detectAndDecodeMulti(img)
        if ok and decoded_info:
            for d in decoded_info:
                if d:
                    return d
    except Exception:
        pass

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    data2, _, _ = detector.detectAndDecode(th)
    return data2 or None


# ------------------- IMAGE SIMILARITY -------------------
def phash_distance(img_path_a, img_path_b):
    """Compute perceptual hash distance between two images (lower = more similar)."""
    a = Image.open(img_path_a)
    b = Image.open(img_path_b)
    ha = imagehash.phash(a)
    hb = imagehash.phash(b)
    return ha - hb

def ssim_score(img_path_a, img_path_b):
    """Compute Structural Similarity Index (SSIM) between two images (0-1 scale)."""
    a = cv2.imread(img_path_a, cv2.IMREAD_GRAYSCALE)
    b = cv2.imread(img_path_b, cv2.IMREAD_GRAYSCALE)
    h = min(a.shape[0], b.shape[0])
    w = min(a.shape[1], b.shape[1])
    a_r = cv2.resize(a, (w, h))
    b_r = cv2.resize(b, (w, h))
    return ssim(a_r, b_r)

def orb_match_score(img_path_a, img_path_b):
    """Compute ORB feature matching score between two images (higher = more similar)."""
    a = cv2.imread(img_path_a, cv2.IMREAD_GRAYSCALE)
    b = cv2.imread(img_path_b, cv2.IMREAD_GRAYSCALE)
    orb = cv2.ORB_create(500)
    kp1, des1 = orb.detectAndCompute(a, None)
    kp2, des2 = orb.detectAndCompute(b, None)
    if des1 is None or des2 is None:
        return 0
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    good = [m for m in matches if m.distance < 60]
    return len(good) / max(1, min(len(kp1), len(kp2)))


# ------------------- QR EVALUATION -------------------
def evaluate_qr(candidate_img, reference_img, expected_payload_substring=None):
    """
    Evaluate a candidate QR image against a reference.
    Returns a dictionary with:
      - decoded payload
      - similarity metrics (phash, SSIM, ORB)
      - risk score
      - suspected flag
    """
    result = {}
    payload = decode_qr(candidate_img)
    result['payload'] = payload

    if not payload:
        result['payload_ok'] = False
        result['payload_reason'] = 'Tidak terbaca'
    else:
        ref_payload = decode_qr(reference_img)
        result['payload_ok'] = (payload == ref_payload)
        result['payload_reason'] = (
            'same payload' if result['payload_ok'] else 'payload differs'
        )

    # Image similarity metrics
    result['phash_dist'] = phash_distance(candidate_img, reference_img)
    result['ssim'] = ssim_score(candidate_img, reference_img)
    result['orb_score'] = orb_match_score(candidate_img, reference_img)

    risk = 0
    if not result['payload_ok']:
        risk += 50
    if result['phash_dist'] > 8:
        risk += 20
    if result['ssim'] < 0.7:
        risk += 15
    if result['orb_score'] < 0.15:
        risk += 15

    result['risk_score'] = risk
    result['is_suspected'] = risk >= 40
    return result


if __name__ == "__main__":
    candidate_file = "/content/drive/MyDrive/ScanQR/altered_qr.png"
    reference_file = "/content/drive/MyDrive/ScanQR/qr2.jpg"

    if not os.path.exists(candidate_file) or not os.path.exists(reference_file):
        raise FileNotFoundError("Candidate or reference QR image not found.")

    expected_id = "ID1019000001403"
    print("Evaluating QR...")

    result = evaluate_qr(candidate_file, reference_file, expected_id)

    print("\n=== QR EVALUATION RESULT ===")
    for k, v in result.items():
        print(f"{k}: {v}")


Evaluating QR...

=== QR EVALUATION RESULT ===
payload: 26650013ID.CO.BCA.WWW011893600014000094045302150008850009404530303UKE51440014ID.CO.QRIS.WWW0215ID20200340731930303UKE5204507253033605802ID5916InterActive Corp6015Permata Safira 61056013662070703A01000201010211630404F3
payload_ok: False
payload_reason: payload differs
phash_dist: 22
ssim: 0.09322572444608174
orb_score: 0.0023148148148148147
risk_score: 100
is_suspected: True


### QRIS Generator

https://twnku.github.io/qrisgen/

## QRIS Editor — How the QR Code Manipulation Works

This website **[QRIS Editor](https://tegarsbl.github.io/QRIS-Editor/)** demonstrates how a malicious actor can **modify (alter) the content of a QR code** used for payments.

### How the scam works

* The attacker creates a **base QR code** that looks identical to the legitimate merchant’s QR code (the victim).
* When the QR code is **altered**, the **visible information** — such as the **merchant’s name** — still appears correct, making it look authentic.
* However, the **payload inside the QR code** (which determines the real payment destination — e.g., acquirer ID or receiving account) is changed.
* As a result, when a customer scans the code, the payment is **redirected to the scammer’s account** instead of the legitimate merchant’s.
* The payer believes they are paying the real merchant because the merchant’s name and branding still appear normal.
